In [28]:
#Libraries
import pandas as pd
import numpy as np
import re
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import cross_val_score

#Training (891 entries) & Testing (418 entries) data
train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')
all_data = [train_data, test_data]

In [2]:
#Feature 1: Pclass
print( train_data[["Pclass","Survived"]].groupby(["Pclass"], as_index = False).mean() )

   Pclass  Survived
0       1  0.629630
1       2  0.472826
2       3  0.242363


In [3]:
#Feature 2: Sex
print( train_data[["Sex", "Survived"]].groupby(["Sex"], as_index = False).mean())

      Sex  Survived
0  female  0.742038
1    male  0.188908


In [4]:
#Feature 3: Family Size
for data in all_data:
    data['family_size'] = data['SibSp'] + data['Parch'] + 1
print( train_data[["family_size","Survived"]].groupby(["family_size"], as_index = False).mean() )

   family_size  Survived
0            1  0.303538
1            2  0.552795
2            3  0.578431
3            4  0.724138
4            5  0.200000
5            6  0.136364
6            7  0.333333
7            8  0.000000
8           11  0.000000


In [5]:
#Feature 3.1: Is alone?
for data in all_data:
    data['is_alone'] = 0
    data.loc[data['family_size'] == 1, 'is_alone'] = 1
print (train_data[['is_alone', 'Survived']].groupby(['is_alone'], as_index=False).mean())

   is_alone  Survived
0         0  0.505650
1         1  0.303538


In [6]:
#Feature 4: Embarked
for data in all_data:
    data['Embarked'] = data['Embarked'].fillna('S')
print( train_data[["Embarked","Survived"]].groupby(["Embarked"], as_index = False).mean() )

  Embarked  Survived
0        C  0.553571
1        Q  0.389610
2        S  0.339009


In [7]:
#Feature 5: Fare
for data in all_data:
    data['Fare'] = data['Fare'].fillna(data['Fare'].median())
train_data['category_fare'] = pd.qcut(train_data['Fare'], 4)
print( train_data[["category_fare","Survived"]].groupby(["category_fare"], as_index = False).mean() )

     category_fare  Survived
0   (-0.001, 7.91]  0.197309
1   (7.91, 14.454]  0.303571
2   (14.454, 31.0]  0.454955
3  (31.0, 512.329]  0.581081


In [8]:
#Feature 6: Age
for data in all_data:
    age_avg  = data['Age'].mean()
    age_std  = data['Age'].std()
    age_null = data['Age'].isnull().sum()

    random_list = np.random.randint(age_avg - age_std, age_avg + age_std , size = age_null)
    data['Age'][np.isnan(data['Age'])] = random_list
    data['Age'] = data['Age'].astype(int)

train_data['category_age'] = pd.cut(train_data['Age'], 5)
print( train_data[["category_age","Survived"]].groupby(["category_age"], as_index = False).mean() )

    category_age  Survived
0  (-0.08, 16.0]  0.522936
1   (16.0, 32.0]  0.356828
2   (32.0, 48.0]  0.370968
3   (48.0, 64.0]  0.434783
4   (64.0, 80.0]  0.090909


/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [9]:
#Feature 7: Name
def get_title(name):
    title_search = re.search(' ([A-Za-z]+)\. ', name)
    if title_search:
        return title_search.group(1)
    return ""

for data in all_data:
    data['title'] = data['Name'].apply(get_title)

for data in all_data:
    data['title'] = data['title'].replace(['Lady', 'Countess','Capt', 'Col','Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'],'Rare')
    data['title'] = data['title'].replace('Mlle','Miss')
    data['title'] = data['title'].replace('Ms','Miss')
    data['title'] = data['title'].replace('Mme','Mrs')
    
print(pd.crosstab(train_data['title'], train_data['Sex']))
print("----------------------")
print(train_data[['title','Survived']].groupby(['title'], as_index = False).mean())

Sex     female  male
title               
Master       0    40
Miss       185     0
Mr           0   517
Mrs        126     0
Rare         3    20
----------------------
    title  Survived
0  Master  0.575000
1    Miss  0.702703
2      Mr  0.156673
3     Mrs  0.793651
4    Rare  0.347826


In [10]:
#Map Data
for data in all_data:

    #Mapping Sex
    sex_map = { 'female':0 , 'male':1 }
    data['Sex'] = data['Sex'].map(sex_map).astype(int)

    #Mapping Title
    title_map = {'Mr':1, 'Miss':2, 'Mrs':3, 'Master':4, 'Rare':5}
    data['title'] = data['title'].map(title_map)
    data['title'] = data['title'].fillna(0)

    #Mapping Embarked
    embark_map = {'S':0, 'C':1, 'Q':2}
    data['Embarked'] = data['Embarked'].map(embark_map).astype(int)

    #Mapping Fare
    data.loc[ data['Fare'] <= 7.91, 'Fare']                            = 0
    data.loc[(data['Fare'] > 7.91) & (data['Fare'] <= 14.454), 'Fare'] = 1
    data.loc[(data['Fare'] > 14.454) & (data['Fare'] <= 31), 'Fare']   = 2
    data.loc[ data['Fare'] > 31, 'Fare']                               = 3
    data['Fare'] = data['Fare'].astype(int)

    #Mapping Age
    data.loc[ data['Age'] <= 16, 'Age']                       = 0
    data.loc[(data['Age'] > 16) & (data['Age'] <= 32), 'Age'] = 1
    data.loc[(data['Age'] > 32) & (data['Age'] <= 48), 'Age'] = 2
    data.loc[(data['Age'] > 48) & (data['Age'] <= 64), 'Age'] = 3
    data.loc[ data['Age'] > 64, 'Age']                        = 4

#Feature Selection
#Create list of columns to drop
drop_elements = ["Name", "Ticket", "Cabin", "SibSp", "Parch", "family_size"]

#Drop columns from both data sets
train_data = train_data.drop(drop_elements, axis = 1)
train_data = train_data.drop(['PassengerId','category_fare', 'category_age'], axis = 1)
test_data = test_data.drop(drop_elements, axis = 1)

#Print ready to use data
print(train_data.head(10))

   Survived  Pclass  Sex  Age  Fare  Embarked  is_alone  title
0         0       3    1    1     0         0         0      1
1         1       1    0    2     3         1         0      3
2         1       3    0    1     1         0         1      2
3         1       1    0    2     3         0         0      3
4         0       3    1    2     1         0         1      1
5         0       3    1    2     1         2         1      1
6         0       1    1    3     3         0         1      1
7         0       3    1    0     2         0         0      4
8         1       3    0    1     1         0         0      3
9         1       2    0    0     2         1         0      3


In [11]:
#Prediction
#Train and Test data
X_train = train_data.drop("Survived", axis=1)
Y_train = train_data["Survived"]
X_test  = test_data.drop("PassengerId", axis=1).copy()

In [57]:
#Running our classifier
decision_tree = DecisionTreeClassifier()
decision_tree.fit(X_train, Y_train)
Y_pred = decision_tree.predict(X_test)
accuracy = round(decision_tree.score(X_train, Y_train) * 100, 2)
print("Model Accuracy: ",accuracy)

Model Accuracy:  87.21


In [58]:
n_splits=10
rstate=15
scoring = 'accuracy'
cv = StratifiedShuffleSplit(n_splits=n_splits, test_size=0.2, random_state=rstate)
score = cross_val_score(decision_tree, X_train, Y_train, cv=cv, scoring='accuracy')
print("Mean CV Accuracy:",cross_val_score(decision_tree, X_train, Y_train, cv=cv, scoring=scoring).mean())



Mean CV Accuracy: 0.8106145251396649


In [59]:
#Create a CSV with results
submission = pd.DataFrame({
    "PassengerId": test_data["PassengerId"],
    "Survived": Y_pred
})
submission.to_csv('submission.csv', index = False)

In [61]:
Y_train

0      0
1      1
2      1
3      1
4      0
5      0
6      0
7      0
8      1
9      1
10     1
11     1
12     0
13     0
14     0
15     1
16     0
17     1
18     0
19     1
20     0
21     1
22     1
23     1
24     0
25     1
26     0
27     0
28     1
29     0
      ..
861    0
862    1
863    0
864    0
865    1
866    1
867    0
868    0
869    1
870    0
871    1
872    0
873    0
874    1
875    1
876    0
877    0
878    0
879    1
880    1
881    0
882    0
883    0
884    0
885    0
886    0
887    1
888    0
889    1
890    0
Name: Survived, Length: 891, dtype: int64

In [19]:
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
target = to_categorical(Y_train)

In [23]:
results = []
#create model
model_2 = Sequential()

#get number of columns in training data
n_cols_2 = X_train.shape[1]

#add layers to model
model_2.add(Dense(250, activation='relu', input_shape=(n_cols_2,)))
model_2.add(Dense(250, activation='relu'))
model_2.add(Dense(250, activation='relu'))
model_2.add(Dense(2, activation='softmax'))

In [24]:
#compile model using accuracy to measure model performance
model_2.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [25]:
results = model_2.fit(X_train, target, epochs=100, batch_size=10, validation_split=0.2)
np.array(results.history['acc']).mean()

Train on 712 samples, validate on 179 samples
Epoch 1/100
712/712 [==============================] - 1s 2ms/step - loss: 0.5178 - acc: 0.7500 - val_loss: 0.4125 - val_acc: 0.8156
Epoch 2/100
712/712 [==============================] - 0s 362us/step - loss: 0.4790 - acc: 0.7781 - val_loss: 0.4022 - val_acc: 0.8380
Epoch 3/100
712/712 [==============================] - 0s 341us/step - loss: 0.4743 - acc: 0.7823 - val_loss: 0.4265 - val_acc: 0.8268
Epoch 4/100
712/712 [==============================] - 0s 436us/step - loss: 0.4583 - acc: 0.7907 - val_loss: 0.3875 - val_acc: 0.8268
Epoch 5/100
712/712 [==============================] - 0s 418us/step - loss: 0.4534 - acc: 0.8034 - val_loss: 0.4318 - val_acc: 0.8268
Epoch 6/100
712/712 [==============================] - 0s 343us/step - loss: 0.4511 - acc: 0.8076 - val_loss: 0.3703 - val_acc: 0.8492
Epoch 7/100
712/712 [==============================] - 0s 341us/step - loss: 0.4380 - acc: 0.8034 - val_loss: 0.3679 - val_acc: 0.8436
Epoch 8/100

Epoch 61/100
712/712 [==============================] - 0s 373us/step - loss: 0.3269 - acc: 0.8581 - val_loss: 0.4171 - val_acc: 0.8492
Epoch 62/100
712/712 [==============================] - 0s 402us/step - loss: 0.3325 - acc: 0.8596 - val_loss: 0.5149 - val_acc: 0.8436
Epoch 63/100
712/712 [==============================] - 0s 331us/step - loss: 0.3259 - acc: 0.8553 - val_loss: 0.5296 - val_acc: 0.8492
Epoch 64/100
712/712 [==============================] - 0s 376us/step - loss: 0.3245 - acc: 0.8610 - val_loss: 0.5080 - val_acc: 0.8547
Epoch 65/100
712/712 [==============================] - 0s 356us/step - loss: 0.3258 - acc: 0.8596 - val_loss: 0.4750 - val_acc: 0.8380
Epoch 66/100
712/712 [==============================] - 0s 384us/step - loss: 0.3250 - acc: 0.8483 - val_loss: 0.4958 - val_acc: 0.8492
Epoch 67/100
712/712 [==============================] - 0s 485us/step - loss: 0.3293 - acc: 0.8581 - val_loss: 0.5059 - val_acc: 0.8436
Epoch 68/100
712/712 [==========================

0.8417696579729907